# Continuous Tomography

This Juypter Notebook is a tutorial for generating simulated data for the publication: ..................

Table of Contents


## 1. Manage Imports 
This section is to manage imports for libraries necessary to run the code. Data will be simulated from the Nanocage 3D volume.

In [ ]:
import tomobase
import numpy as np
import tomobase.phantoms
import tomobase.processes
import tomobase.processes.alignments
import tomobase.tiltschemes
import pandas as pd

from skimage.metrics import peak_signal_noise_ratio, structural_similarity
vol = tomobase.phantoms.nanocage()
pd.set_option('display.max_rows', None)

## 2. TiltScheme Setup
The following section setups a loop to acquire all base tilt scheme angles required in this study

In [ ]:

# return all tiltschemes for basic tests
def generate_schemes(scheme_selection=None):
    schemes = []
    labels = []
    angles = []
    
    if scheme_selection is None:
        scheme_selection = [0,1,2]
    
    for set in scheme_selection: 
        match set:
            case 0:
                incremental_schemes = [tomobase.tiltschemes.Incremental(-64, 64, 2), 
                                        tomobase.tiltschemes.Incremental(-64, 64, 4), 
                                        tomobase.tiltschemes.Incremental(-64, 64, 8), 
                                        tomobase.tiltschemes.Incremental(-64, 64, 16)]
                incremental_angles = [65, 33, 17, 9, 5]
                incremental_labels = ['Incremental', 'Incremental', 'Incremental', 'Incremental', 'Incremental']
                schemes.extend(incremental_schemes)
                angles.extend(incremental_angles)
                labels.extend(incremental_labels)
            case 1:
                binary_schemes = [tomobase.tiltschemes.Binary(-64, 64,k=2), 
                                    tomobase.tiltschemes.Binary(-64, 64,k=2), 
                                    tomobase.tiltschemes.Binary(-64, 64,k=2),
                                    tomobase.tiltschemes.Binary(-64, 64,k=2),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8), 
                                    tomobase.tiltschemes.Binary(-64, 64,k=2, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=2, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=2, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=2, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8, isbidirectional=False)]
                binary_angles = [65, 33, 17, 9, 65, 33, 17, 9, 65, 33, 17, 9, 65, 33, 17, 9, 65, 33, 17, 9, 65, 33, 17, 9]
                binary_labels = ['Binary 2b', 'Binary 2b', 'Binary 2b', 'Binary 2b', 'Binary 4b', 'Binary 4b', 'Binary 4b', 'Binary 4b', 'Binary 8b', 'Binary 8b', 'Binary 8b', 'Binary 8b', 'Binary 2u', 'Binary 2u', 'Binary 2u', 'Binary 2u', 'Binary 4u', 'Binary 4u', 'Binary 4u', 'Binary 4u', 'Binary 8u', 'Binary 8u', 'Binary 8u', 'Binary 8u']
                
                schemes.extend(binary_schemes)
                angles.extend(binary_angles)
                labels.extend(binary_labels)
            case 2:
                grs_schemes =  [tomobase.tiltschemes.GRS(-64, 64,0),
                                tomobase.tiltschemes.GRS(-64, 64,0),
                                tomobase.tiltschemes.GRS(-64, 64,0),
                                tomobase.tiltschemes.GRS(-64, 64,0)]
                grs_angles = [65, 33, 17, 9]
                grs_labels = ['GRS', 'GRS', 'GRS', 'GRS']
                schemes.extend(grs_schemes)
                angles.extend(grs_angles)
                labels.extend(grs_labels)
    return schemes, angles, labels


## Tests
The following section outlines the tests performed in this study

Table of Contents
[3.1. Display Angles](#31-display-angles) - Displays the angles acquired for novel schemes

### 3.1. Display Angles
The purpose of this test is just to be able to read off the angles for the binary tiltschemes. Two almost identical scheme sets are presented:

1. Unidirectional (u) -  once the highest angle in the set is reached the tiltseries is reverted back to the minimum angle with an offset.
2. Bidirectional (b) -  Once the highest angle in the set is reached the tiltseries is offset than angles are collected backwards.

Both schemes should have collected the exact same angles in a set of k+1 projections - just in a different order. 

In [ ]:

schemes, n_angles, labels = generate_schemes([1])
coloumns = []

for i in range(len(schemes)):
    if n_angles[i]==65:
        print(schemes[i].k, schemes[i].isbidirectional)
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])])
        coloumns.append(angles)

_dict = {
    'Binary 2b': coloumns[0],
    'Binary 4b': coloumns[1],
    'Binary 8b': coloumns[2],
    'Binary 2u': coloumns[3],
    'Binary 4u': coloumns[4],
    'Binary 8u': coloumns[5],
}
df = pd.DataFrame(_dict)

display(df)

### 3.2. Backlash

There is an artifact in the microscope gonioometer for bidirectional acquisition schemes. This section outlines its effects on the different tiltschemes



In [ ]:
schemes, n_angles, labels = generate_schemes()
backlash_value = 0.1

banned = [0, 1, 2, 3, 4, 5, 6]

df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'Backlash Value', 'Cumulated Backlash', 'Correction', 'SSIM without error', 'PSNR without error', 'SSIM with error', 'PSNR with error', 'SSIM corrected', 'PSNR corrected'])
for i, scheme in enumerate(schemes):
    if i not in banned:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
        sino = tomobase.processes.project(vol, angles)
        indices = np.where(np.diff(sino.angles) < 0)[0] + 1
        
        rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
        ssim_without_error = structural_similarity(vol.data, rec.data, data_range=1.0)
        psnr_without_error = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
        
        cumulated_backlash = backlash_value * len(indices)
        sino.angles[indices] -= backlash_value
        
        rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
        ssim_with_error = structural_similarity(vol.data, rec.data, data_range=1.0)
        psnr_with_error = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
         
        sino, correction = tomobase.processes.alignments.backlash_correct(sino, extend_return=True)
        rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
        
        ssim_corrected = structural_similarity(vol.data, rec.data, data_range=1.0)
        psnr_corrected = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)

        new_row = {
            'Tilt Scheme': labels[i],
            'Number of Angles': n_angles[i],
            'Backlash Value': backlash_value,
            'Cumulated Backlash': cumulated_backlash,
            'Correction': correction,
            'SSIM without error': ssim_without_error,
            'PSNR without error': psnr_without_error,
            'SSIM with error': ssim_with_error,
            'PSNR with error': psnr_with_error,
            'SSIM corrected': ssim_corrected,
            'PSNR corrected': psnr_corrected
        }

        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

print(df) 

In [ ]:
schemes, n_angles, labels = generate_schemes()
backlash_value = 0.5

banned = [0, 1, 2, 3, 4]

df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'Backlash Value', 'Cumulated Backlash', 'Correction', 'MAE without error', 'MAE with error', 'MAE corrected'])
for i, scheme in enumerate(schemes):
    if i not in banned:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
        sino = tomobase.processes.project(vol, angles)
        indices = np.where(np.diff(sino.angles) < 0)[0] + 1
        
        rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
        mae_without_error = np.mean(np.abs(vol.data - rec.data))
        
        cumulated_backlash = backlash_value * len(indices)
        sino.angles[indices] -= backlash_value
        
        if len(indices) > 0:
            rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
            mae_with_error = np.mean(np.abs(vol.data - rec.data))
            
            sino, correction = tomobase.processes.alignments.backlash_correct(sino, extend_return=True)
            rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
            mae_corrected = np.mean(np.abs(vol.data - rec.data))
        else:
            mae_with_error = mae_without_error
            mae_corrected = mae_without_error
            correction = 0

        new_row = {
            'Tilt Scheme': labels[i],
            'Number of Angles': n_angles[i],
            'Backlash Value': backlash_value,
            'Cumulated Backlash': cumulated_backlash,
            'Correction': correction,
            'MAE without error': mae_without_error,
            'MAE with error': mae_with_error,
            'MAE corrected': mae_corrected
        }

        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

print(df)

In [ ]:
print(df.to_csv(index=False))

### Missing Wedge

In [ ]:
schemes, n_angles, labels = generate_schemes()

banned = []

df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'Missing Wedge'])
for i, scheme in enumerate(schemes):
    if i not in banned:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
        missing_wedge = 180 - (np.max(angles) - np.min(angles))


        new_row = {
            'Tilt Scheme': labels[i],
            'Number of Angles': n_angles[i],
            'Missing Wedge': missing_wedge,
        }

        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

display(df)

In [ ]:
schemes = [tomobase.tiltschemes.Incremental(-64, 64, 32), 
           tomobase.tiltschemes.Incremental(-64, 64, 16),
           tomobase.tiltschemes.Incremental(-64, 64, 8),
           tomobase.tiltschemes.Incremental(-64, 64, 4),
           tomobase.tiltschemes.Incremental(-64, 64, 2)]

n_angles = [5, 9, 17, 33, 65]
array = np.zeros((len(schemes), 3))
for i, item in enumerate(schemes):
    array[i,0], array[i,1], array[i,2] = process_calculate_psnr_ssim(item, n_angles[i])
    
for i, item in enumerate(schemes):
    print(array[i,0], array[i,1], array[i,2])

### GRS

Loop for performing GRS acquisition and analysis

In [ ]:
num = 65
ssim = [None]*num
psnr = [None]*num
index = [None]*num

cumulation = 0
array = np.zeros((num, 3))
for i in range(num):
    scheme = tomobase.tiltschemes.GRS(-64, 64)
    angles = np.array([scheme.get_angle() for j in range(i+1)])
    array[i,0], array[i,1], array[i,2] = quantify_backlash(scheme, i+1, cumulation)
    cumulation = array[i,2]
        
for i in range(len(index)):    
    print(array[i,0], array[i,1], array[i,2])

### Binary

In [ ]:
schemes, labels, n= generate_schemes()
scheme = schemes[7]
#scheme = tomobase.tiltschemes.Binary(-64, 64, 8)
angles = np.array([scheme.get_angle() for j in range(65)])
sino = tomobase.processes.project(vol, angles)

indices = np.where(np.diff(sino.angles) < 0)[0] + 1
sino.angles[indices] -= 0
rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
ssim_uncorrected = structural_similarity(vol.data, rec.data, data_range=1.0)
psnr_uncorrected = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)


indices = np.where(np.diff(sino.angles) < 0)[0] + 1
sino.angles[indices] -= 0.1
sino, correction = tomobase.processes.alignments.backlash_correct(sino, extend_return=True)
rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
ssim_corrected = structural_similarity(vol.data, rec.data, data_range=1.0)
psnr_corrected = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)

print(correction, ssim_uncorrected, psnr_uncorrected, ssim_corrected, psnr_corrected) 